# AdventureWorks Försäljningsanalys

I denna analys undersöker vi AdventureWorks-databasen för att besvara affärsfrågor relaterade till försäljning och kundbeteende, med fokus på områden som:
- Produktkategorier och deras försäljning
- Försäljningstrender över tid
- Regional försäljning

SQL användes för att hämta data, medan visualisering och analys utfördes med hjälp av Python.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

user = "SA"
password = quote_plus("Makedonien00")
server = 'localhost:1433'
database = 'AdventureWorks2025'
driver = quote_plus("ODBC Driver 18 for SQL server")

connection_string = (
    f"mssql+pyodbc://{user}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)


try:
    with engine.connect():
        print("Anslutning till SQL Server lyckades")
except Exception as e:
    print("Kunde inte ansluta", e)

In [ ]:
def query_df(sql: str):
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)

## Visualisering 1: Antal produkter per kategori
Hur många produkter finns i varje kategori?

Tabeller som används:
- Production.ProductCategory
- Production.ProductSubCategory
- Production.Product

Plan:
- JOINA tabellerna
- Räkna DISTINCT produkter per kategori
- Skapa vertikalt stapeldiagram
- Analysera resultatet


In [ ]:
query_vis1 = """
SELECT
    pc.Name AS CategoryName,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubCategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubCategoryID = p.ProductSubCategoryID
GROUP BY pc.Name
ORDER BY ProductCount DESC
"""

df_vis1 = query_df(query_vis1)



fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(df_vis1['CategoryName'], df_vis1['ProductCount'], color='steelblue', alpha=0.8)

for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{int(height)}',
        ha='center',
        va='bottom'
    )
ax.set_xlabel('Produktkategori', fontsize=12)
ax.set_ylabel('Antal produkter', fontsize=12)
ax.set_title('Antal produkter per kategori', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Insikter - Antal produkter per kategori

Components är största kategorin med 134 produkter.
Accessories är minst med 29 produkter.

Detta indikerar på att företaget är starkt fokuserat på komponenter och reservdelar, även att accessories kan vara ett utvecklingsområde.

## Visualisering 2: Försäljning per produktkategori
Vilka produktkategorier genererar mest intäkter?

Tabeller som används:
- Production.ProductCategory
- Production.ProductSubcategory
- Production.Product
- Sales.SalesOrderDetail

Plan:
- JOINA tabellerna
-Räkna total försäljning per kategori med SUM
- Skapa horisontellt stapeldiagram
- Analysera resultatet


In [ ]:
query_vis2 = """

SELECT
    pc.Name AS CategoryName,
    SUM(sod.LineTotal) AS TotalSales
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubCategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubCategoryID = p.ProductSubCategoryID
INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
GROUP BY pc.Name
ORDER BY TotalSales DESC
""" 

df_vis2 = query_df(query_vis2)
df_vis2.sort_values('TotalSales', inplace=True, ascending=True)

fig, ax = plt.subplots(figsize=(20, 10))

bars = ax.barh(df_vis2['CategoryName'], df_vis2['TotalSales'], color='steelblue', alpha=0.8)
for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height() / 2,
        f'{width:,.0f}',
        ha='left',
        va='center'
    )
ax.set_ylabel('Produktkategori', fontsize=12)
ax.set_xlabel('Total försäljning (kr)', fontsize=12)
ax.set_title('Total försäljning per produktkategori', fontsize=14, fontweight='bold')

ax.set_xticklabels([f'{x:,.0f}' for x in ax.get_xticks()])


plt.tight_layout()
plt.show() 

### Insikter - Försäljning per produktkategori

Bikes är kategorin med störst total försäljning på 94 651 173kr. Accessories är kategorin med minst total försäljning på 1 272 073kr. 

Detta indikerar på att bikes, som är huvudprodukten, säljer mest. Accessories totala försäljning kan bero på att produkterna har lägre prisnivåer men även en lägre efterfrågan.

## Visualisering 3: Försäljningstrend över tid
Hur har försäljningen utvecklats över tid?

Tabeller som används:
- Sales.SalesOrderHeader

Plan:
- Räkna total försäljning per månad och år med SUM
- Gruppera efter år och månad med GROUP BY 
- Skapa linjediagram
- Analysera resultatet

In [ ]:
query_vis3 = """

SELECT
    SUM(TotalDue) AS TotalOrderSales,
    YEAR(OrderDate) AS OrderYear,
    MONTH(OrderDate) AS OrderMonth
FROM Sales.SalesOrderHeader
GROUP BY
    YEAR(OrderDate),
    MONTH(OrderDate)
ORDER BY OrderYear , OrderMonth;
"""


df_vis3 = query_df(query_vis3)

df_vis3['OrderMonth'] = pd.to_datetime(df_vis3['OrderYear'].astype(str) + '-' + df_vis3['OrderMonth'].astype(str))

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(
    df_vis3['OrderMonth'],
    df_vis3['TotalOrderSales'],
    marker='o',
    color='steelblue' 
)

ax.set_title('Försäljningstrend per månad', fontsize=14, fontweight='bold')
ax.set_ylabel('Total försäljning (kr)', fontsize=12)

ax.set_yticklabels([f'{x:,.0f}' for x in ax.get_yticks()])

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Insikter -Försäljningstrend per månad

Försäljningstrenden har generellt ökat under åren, fram tills våren 2025 där det skedde en rejäl minskning, april 2025 till juni 2025. 
Högsta månaden är april 2025.
Lägsta månaden är juni 2025.

Det verkar inte finnas en tydlig säsongsberoende trend.

## Visualisering 4: Försäljning och antal ordrar per år
Hur ser total försäljning och antal ordrar ut per år?

Tabeller som används:
- Sales.SalesOrderHeader

Plan:
- Summera total försäljning per år med SUM
- Beräkna antal ordrar med COUNT
- Gruppera efter år med GROUP BY
- Skapa ett grupperat stapeldiagram
- Analysera resultatet

In [ ]:
query_vis4 = """

SELECT
    SUM(TotalDue) AS TotalOrderSales,
    COUNT(SalesOrderID) AS OrderCount,
    YEAR(OrderDate) AS OrderYear
FROM Sales.SalesOrderHeader
GROUP BY
    YEAR(OrderDate)
ORDER BY OrderYear;
"""

df_vis4 = query_df(query_vis4)


x = range(len(df_vis4))
fig, ax1 = plt.subplots(figsize=(10, 6))
width = 0.4



ax1.bar(
    [i - width/2 for i in x],
    df_vis4['TotalOrderSales'],
    color='steelblue',
    label='Total försäljning (kr)',
    width=width,
    alpha = 0.8
)
ax1.set_ylabel('Total försäljning (kr)', fontsize=12)

ax2 = ax1.twinx()
ax2.bar(
    [i + width/2 for i in x],
    df_vis4['OrderCount'],
    color='orange',
    label='Antal ordrar',
    width=width,
    alpha = 0.8
)
ax2.set_ylabel('Antal ordrar', fontsize=12)



ax1.set_title('Försäljning och antal ordrar per år', fontsize = 14, fontweight= 'bold')

ax1.set_xticks(x)
ax1.set_xticklabels(df_vis4['OrderYear'], rotation=45, ha='right')

ax1.set_yticklabels([f'{x:,.0f}' for x in ax1.get_yticks()])


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')


plt.tight_layout()
plt.show()

### Insikter - Försäljning och antal ordrar per år

Fram tills 2024 ökade både försäljning och antal ordrar i takt med varandra.
År 2024 hade flest antal ordrar samt högst försäljning. Från 2024 till 2025 minskar försäljningen kraftigt, medan antalet ordrar minskar marginellt. 

## Visualisering 5: Top 10 produkter efter försäljning
Vilka 10 produkter genererar mest försäljning?

Tabeller som används:
- Production.Product
- Sales.SalesOrderDetail

Plan:
- Begränsa antal med TOP 10
- JOINA tabellerna 
- Räkna total försäljning per produkt med SUM
- Gruppera efter produktnamn med GROUP BY
- Skapa hosisontellt stapeldiagram
- Analysera resultatet


In [ ]:
query_vis5  = """

SELECT TOP 10
    p.Name AS ProductName,
    SUM(sod.LineTotal) AS TotalSales
FROM Production.Product p 
INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
GROUP BY p.Name
ORDER BY TotalSales DESC;
"""

df_vis5 = query_df(query_vis5)
df_vis5.sort_values('TotalSales', inplace=True, ascending=True)

fig, ax = plt.subplots(figsize=(20, 10))
bars = ax.barh(df_vis5['ProductName'], df_vis5['TotalSales'], color='steelblue', alpha=0.8)

for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height() / 2,
        f'{width:,.0f}',
        ha='left',
        va='center'
    )
ax.set_ylabel('Produkter', fontsize=12)
ax.set_xlabel('Total försäljning (kr)', fontsize=12)
ax.set_title('Top 10 produkter efter försäljning', fontsize=14, fontweight='bold')

ax.set_xticklabels([f'{x:,.0f}' for x in ax.get_xticks()])

plt.tight_layout()
plt.show()


### Insikter - Top 10 produkter efter försäljning

Mountain-200 Black, 38 är den mest sålda produkten med en försäljning på 4 400 594kr. 

Topp 10 domineras av kategorin Bikes, som indikerar att cyklar står för majoriteten av försäljningsintäkterna.

## Visualisering 6: Försäljning och antal kunder per region
Hur skiljer sig försäljningen mellan olika regioner, och hur många unika kunder har varje region?

Tabeller som används: 
- Sales.SalesTerritory
- Sales.SalesOrderHeader
- Sales.Customer

Plan:
- JOINA tabellerna
- Summera total försäljning per region med SUM
- Beräkna antal kunder per region med COUNT
- Gruppera efter region med GROUP BY
- Sortera från högst till lägst försäljning med DESC
- Skapa grupperat stapeldiagram
- Analysera resultatet

In [ ]:
query_vis6 = """

SELECT
    st.Name AS Territory,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(DISTINCT c.CustomerID) AS Customers
FROM Sales.SalesTerritory st
INNER JOIN Sales.SalesOrderHeader soh ON st.TerritoryID = soh.TerritoryID
INNER JOIN Sales.Customer c ON soh.CustomerID = c.CustomerID
GROUP BY st.Name
ORDER BY TotalSales DESC;
"""

df_vis6 = query_df(query_vis6)

x = range(len(df_vis6))
width = 0.4

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(
    [i - width/2 for i in x],
    df_vis6['TotalSales'],
    width=width,
    color='steelblue',
    label='Total försäljning (kr)',
    alpha=0.8
)
ax1.set_ylabel('Total försäljning (kr)', fontsize=12)

ax2 = ax1.twinx()
ax2.bar(
    [i + width/2 for i in x],
    df_vis6['Customers'],
    width=width,
    color='orange',
    label='Antal kunder',
    alpha=0.8
)
ax2.set_ylabel('Antal kunder', fontsize=12)

ax1.set_xticks(list(x))
ax1.set_xticklabels(df_vis6['Territory'], rotation=45, ha='right')


ax1.set_title('Försäljning och antal kunder per region', fontsize=14, fontweight='bold')

ax1.set_yticklabels([f'{x:,.0f}' for x in ax1.get_yticks()])


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

plt.tight_layout()
plt.show()

### Insikter - Försäljning och antal kunder per region

Southwest är den starkaste regionen med både flest kunder och högst försäljning, vilket visar att antalet kunder har en stor påverkan på försäljningen i den regionen. 

Germany är den svagast regionen gällande total försäljning, trots att de har relativt många kunder. Detta indikerar på ett lägre genomsnittligt ordervärde.

## Visualisering 7: Genomsnittligt ordervärde per region och kundtyp
Vilka regioner har högst/lägst genomsnittligt ordervärde, och skiljer det sig mellan individuella kunder och företagskunder?

Tabeller som används: 
- Sales.SalesTerritory
- Sales.SalesOrderHeader
- Sales.Customer
- Sales.Store

Plan: 
- JOINA Tabellerna
- Gör CASE för att skilja på Store och Individuella kunder
- Beräkna genomsnittligt ordervärde per kundtyp och region 
- Gruppera efter region och kundtyp med GROUP BY
- Sortera från högst till lägst genomsnitt med DESC
- Skapa grupperat stapeldiagram
- Analysera resultatet

In [ ]:
query_vis7 = """


SELECT
    st.Name AS Territory,
    CASE
        WHEN c.StoreID IS NOT NULL THEN 'Store'
        WHEN c.PersonID IS NOT NULL THEN 'Individual'
    END AS CustomerType,
    SUM(soh.TotalDue) / COUNT(DISTINCT soh.SalesOrderID) AS AverageOrderValue
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st  ON soh.TerritoryID = st.TerritoryID
INNER JOIN Sales.Customer c ON soh.CustomerID = c.CustomerID
LEFT JOIN Sales.Store s ON c.StoreID = s.BusinessEntityID
GROUP BY st.Name,
    CASE
        WHEN c.StoreID IS NOT NULL THEN 'Store'
        WHEN c.PersonID IS NOT NULL THEN 'Individual'
    END
ORDER BY AverageOrderValue DESC;
"""

df_vis7 = query_df(query_vis7)

order= (df_vis7.groupby('Territory')['AverageOrderValue'].mean().sort_values(ascending=False).index)
store_data = df_vis7[df_vis7['CustomerType'] == 'Store'].set_index('Territory').loc[order].reset_index()
individual_data = df_vis7[df_vis7['CustomerType'] == 'Individual'].set_index('Territory').loc[order].reset_index()

x = range(len(store_data))
width = 0.4

fig, ax = plt.subplots(figsize=(12, 6))

ax.bar( 
    [i - width/2 for i in x],
    store_data['AverageOrderValue'],
    width=width,
    color='steelblue',
    label='Store',
    alpha=0.8
)

ax.bar(
    [i + width/2 for i in x],
    individual_data['AverageOrderValue'],
    width=width,
    color='orange',
    label='Individual',
    alpha=0.8
)

ax.set_xticks(list(x))
ax.set_xticklabels(store_data['Territory'], rotation=45, ha='right')
ax.set_ylabel('Genomsnittligt ordervärde (kr)', fontsize=12)

ax.set_xlabel('Försäljningsregion', fontsize=12)
ax.set_title('Genomsnittligt ordervärde per region och kundtyp', fontsize=14, fontweight='bold')

ax.set_yticklabels([f'{x:,.0f}' for x in ax.get_yticks()])

ax.legend()
plt.tight_layout()
plt.show()

## Insikter - Genomsnittligt ordervärde per region och kundtyp

Store har högre genomsnittligt ordervärde i varje region. Southwest och Store har det högsta genomsnittliga ordervärdet medan Australia har det lägsta inom kundtypen Store.

Detta indikerar att företagskunder generellt gör störree ordrar än privatkunder, oavsett vilken region det är.

## Sammanfattning

Huvudsakliga fynd
Rekommendationer - Vad kan företaget göra baserat på denna analys?

Produktkategorier och deras försäljning
- Components är den största kategorin med 134 produkter, medan Accessories är minst med 29 produkter. 
- Största försäljningen utgörs av kategorin Bikes som även dominerar topp 10, medan Accessories har lägst.
- Mountain-200 Balck, 38 är den mest sålda produkten.

Försäljningstrender över tid
- Försäljningen ökade från 2022 till 2024 men minskade kraftigt från 2024 till 2025. 
- Bästa försäljningsmånaden var April 2025 medan juni 2025 hade lägst försäljning.
- 2024 hade flest antal ordrar samt högst försäljning, dock minskade försäljningen markant från 2024 till 2025.

Regional försäljning
- Southwest är den starkaste regionen med flest kunder och högst ordervärde. 
- Germany är den svagaste regionen i försäljning, trots många kunder.
- Företagskunder gör generellt stöte ordrar än privata kunder. 


#### Rekommendationer: 
Produktkategorier och deras försäljning
- Fokusera på att skapa och utveckla fler produkter inom vissa kategorier som inte säljer lika bra, som Accessories. Detta kan inkludera att bredda produktutbudet eller skapa fler marknadsföringskampanjer för att öka efterfrågan.

Försäljningtrender över tid
- Undersöka orsakerna bakom nedgången i försäljningen från 2024 till 2025, om det är på grund av marknadsföring, kundbeteende eller säsongsbundna faktorer.
- Marknadsför produkter kopplat till säsonger för att skapa en stabliare försäljning under året. Exempelvis göra vår-, sommar- eller julkampanjer för att öka intresset.
- Erbjuda fler kampanjer, paketpriser eller rabatter som 3 för 2 på vissa produkter för att försöka öka det genomsnittliga ordervärdet som minskade 2024 till 2025.

Regional försäljning
- Stärka försäljning i regioner med lågt genomsnittligt ordervärde, exempelvis Germany som har många kunder men lägre  ordervärde. Därmed kan man fokusera på att marknadsföra lönsammare produkter med högre pris som Bikes, för att höja ordervärdet.
- Anpassa marknadsföring av produkter efter kunder och regionernas olika behov. 